# SUBMISSION FOR DATA ENGINEERING CODING ROUND - TOPOS
  
*Author: Ajinkya Sheth  
University of Washington, Seattle  
ajinkya@uw.edu*

#### Rubric
Topos is using AI to develop a holistic understanding of location. We are looking for engineers who are curious and creative in the way they approach data. Assignment tests your ability to move from an open-ended prompt to a result that illuminates a facet (or facets) of the data that you find intriguing. We will be reviewing your work to understand the way you approach problems generally, the way you write code, and your creativity working with data.

#### Prompt
The goal of this assignment is to demonstrate your ability to capture unconventional datasets, clean and store them.

Write a scraper in either python or NodeJS to collect data from Wikipedia about the top cities in the United States. The fields you collect, as well as the volume of data is up to you, but ideally you add additional data beyond the initial table, such as data found on the individual city pages, or other sources of your choice. The final format should be a CSV file that is ready to be uploaded to a BigQuery table. Please read Bigquery’s Manual to prepare your CSV in the right format. Intermediary steps, environments or processes necessary to run the scraper should be documented in code as well as a Readme.md and hosted on github in a repo devoted to this assignment


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request

## 1. Introduction
This web-scraper is my submission to Topos coding round for the Data Engineering Internship.  
It fetches the population data of top 314 U.S. Cities from Wikipedia.  
To be specific, from the below link:  
https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population

Additionally, it fetches the historical population trend from each of the top 314 cities by scraping the sub-links from the above url.  

There are some cities (for example: Anaheim, California) where the historical population is not present in the Wikipedia webpage. In such cases, the program will notify the user.  

Ultimately, we get two tables:  
1. cities_main which contains population of top 314 cities
2. cities_hist which contains historical population trend of top 314 cities


## 2. cities_main table
Fetches the data from https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population  
Snapshot below:  

![alt text](main.png "List of cities by population")

### 2.1 Set-up BeautifulSoup

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
domain='https://en.wikipedia.org'
req = urllib.request.urlopen(url)
article = req.read().decode()

# Load article, turn into soup and get the <table>s.
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='wikitable sortable')

### 2.2 Get table headings

In [3]:
# Search through the tables for the one with the headings we want.
ths = tables[0].find_all('th')
headings=[]
for th in ths:
    colspan=1
    if th.attrs:
        colspan=int(th.attrs['colspan'])
    header_text=th.text.strip()
    if th.find('sup'):
        header_text=header_text.replace(th.find('sup').text.strip(),"")
    if colspan!=1:
        for i in range(colspan):
            headings.append(header_text+" "+str(i+1))
    else:
        headings.append(header_text)
headings.append('Link')

### 2.3 Get Table Data

In [4]:
table_data=[]
for tr in tables[0].find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue
    row=[]
    for td in tds:
        td_text=td.text.strip()
        if td.find('sup'):
            td_text=td_text.replace(td.find('sup').text.strip(),"")
        row.append(td_text)
    row.append(domain+tds[1].find('a').get('href'))
    table_data.append(row)

### 2.4 Create a dataframe from table_data and headings

In [5]:
cities_main=pd.DataFrame(table_data,columns=headings)

### 2.5 Clean and Rearrange

In [6]:
'''
Function: stripAlpha
Parameters:
    dataframe: dataframe 
    column_list: List of column names
Returns:
    nothing
Description:
    strips non-numeric characters from entire columns in a dataframe
    modifies the dataframe directly
'''
def stripAlpha(dataframe,column_list):
    for col in column_list:
        if col in list(dataframe):
            dataframe[col]=dataframe[col].str.replace('[^0-9]','')
        else:
            print(str(col)+' not found')

In [7]:
rename_dict={"2016 land area 1":"2016 land area (sq miles)",
            "2016 land area 2":"2016 land area (sq km)",
            "2016 population density 1":"2016 population density (sq miles)",
            "2016 population density 2":"2016 population density (sq km)"}

In [8]:
cities_main=cities_main.rename(columns=rename_dict)

In [9]:
stripAlpha(cities_main,['2018estimate','2010Census','2016 land area (sq miles)', '2016 land area (sq km)',
 '2016 population density (sq miles)',
 '2016 population density (sq km)',])

In [10]:
rearrange_column=['City','2018rank','State','2018estimate','2010Census','Change','2016 land area (sq miles)',
'2016 land area (sq km)','2016 population density (sq miles)','2016 population density (sq km)','Location','Link']
cities_main=cities_main[rearrange_column]

In [11]:
cities_main.head()

,City,2018rank,State,2018estimate,2010Census,Change,2016 land area (sq miles),2016 land area (sq km),2016 population density (sq miles),2016 population density (sq km),Location,Link
0,New York City,1,New York,8398748,8175133,+2.74%,3015,7809,28317,10933,40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...,https://en.wikipedia.org/wiki/New_York_City
1,Los Angeles,2,California,3990456,3792621,+5.22%,4687,1139,8484,376,34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...,https://en.wikipedia.org/wiki/Los_Angeles
2,Chicago,3,Illinois,2705994,2695598,+0.39%,2273,5887,11900,4600,41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...,https://en.wikipedia.org/wiki/Chicago
3,Houston,4,Texas,2325502,2100263,+10.72%,6375,16511,3613,1395,29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...,https://en.wikipedia.org/wiki/Houston
4,Phoenix,5,Arizona,1660272,1445632,+14.85%,5176,13406,3120,100,33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...,"https://en.wikipedia.org/wiki/Phoenix,_Arizona"


## 3. cities_hist tables
Visits the individual wikipedia links of cities from 'Link' column of cities_main and fetches the historical population trend.  The picture below displays population trend of Mesa, Atlanta, one of the top 314 cities from https://en.wikipedia.org/wiki/Mesa,_Arizona

![alt text](sub.png "Population Trend of Mesa, Atlanta")


In [12]:
'''
Function: get_citytable
Parameters:
    city_url: string 
Returns:
    BeautifulSoup object
Description:
    scrapes BeautifulSoup object representing html table with class 'toccolours'  
    and row text 'Historical Population' from the wikipedia url)
'''
def get_citytable(city_url):
    url = city_url
    req = urllib.request.urlopen(url)
    article = req.read().decode()

    soup = BeautifulSoup(article, 'html.parser')
    tables = soup.find_all('table', class_='toccolours')
    tab=[]
    for table in tables:
        if table.find('tr').text.strip()=='Historical population':
            tab=table
            break
    return tab



In [13]:
'''
Function: get_cityheaders
Parameters:
    table: BeautifulSoup Object
Returns:
    table headers list
Description:
    scrapes table headers list from BeautifulSoup object representing html table with class 'toccolours'  
    and row text 'Historical Population' from the wikipedia url)
'''
def get_cityheaders(table):
    ths = table.find_all('tr')[1].find_all('th')
    headers=[]
    for th in ths:
        headers.append(th.text.strip())
    for elem in range(len(headers)):
        if headers[elem]=='Census':
            headers[elem]='Year'
        if headers[elem]=='%±':
            headers[elem]='±%'
    return headers

In [14]:
'''
Function: get_city_histdata
Parameters:
    table: BeautifulSoup Object
Returns:
    list of list
Description:
    scrapes table data list from BeautifulSoup object representing html table with class 'toccolours'  
    and row text 'Historical Population' from the wikipedia url)
'''
def get_city_histdata(table):
    table_data=[]
    trs=table.find_all('tr')
    for tr in trs[2:]:
        #colspan=int(tr.attrs['colspan'])
        elems=tr.find_all()
        if not elems:
            continue
        row=[]
        for elem in elems:
            if elem.find('a'):
                ''#print(elem.text.strip())
            else:
                elem_text=elem.text.strip()
                if elem.find('sup'):
                    elem_text=elem_text.replace(elem.find('sup').text.strip(),"")
                row.append(elem_text)
        table_data.append(row)
    return table_data[:-2]

In [15]:
'''
Function: get_city_hist_df
Parameters:
    city_name: Name of the city
    headers: headers list
    data: historical data list of lists
Returns:
    dataframe
Description:
    combines headers and data into a single dataframe
    adds another column city_name to the data frame
'''
def get_city_hist_df(city_name,headers,table_data):
    city_hist=pd.DataFrame(table_data,columns=headers)
    city_hist['city_name']=city_name
    return city_hist

### 3.1 Historical Population scraping utility
This program fetches links and city_name from the cities_main table and fetches 'Historical population' from the respective  
In case, the the information is missing, the scraping utility will print the city name for which the population is not found

In [16]:
cities_hist=pd.DataFrame()
for i in range(314):
    city_name=cities_main[['City','Link']].iloc[i]['City']
    'print(city_name)'
    link=cities_main[['City','Link']].iloc[i]['Link']
    city_table=get_citytable(link)
    if city_table:
        table_headers=get_cityheaders(city_table)
        table_data=get_city_histdata(city_table)
        city_hist=get_city_hist_df(city_name,table_headers,table_data)
        cities_hist=pd.concat([cities_hist,city_hist], ignore_index=True, sort=True)
    else:
        print('Historical Population not found for '+str(city_name))
    

Historical Population not found for Anaheim
Historical Population not found for Newark
Historical Population not found for Cedar Rapids
Historical Population not found for Clinton


### 3.2 Cleaning and Rearrangement

In [17]:
stripAlpha(cities_hist,['Pop.'])            

In [18]:
cities_hist=cities_hist.drop(columns='')

In [19]:
cities_hist=cities_hist[['city_name', 'Year', 'Pop.', '±%']]

In [20]:
cities_hist.head()

,city_name,Year,Pop.,±%
0,New York City,1698,4937,—
1,New York City,1712,5840,+18.3%
2,New York City,1723,7248,+24.1%
3,New York City,1737,10664,+47.1%
4,New York City,1746,11717,+9.9%


## 4. Save csv
Save the two tables into the csv format
The tables can be loaded into Bigtable of GCP and can be merged joined based on the city_name key

In [21]:
cities_hist.to_csv('cities_hist.csv')

In [24]:
cities_main.to_csv('cities_main.csv', encoding='utf-8')